# ECAN Attention Allocation Lab

## Economic Attention Network for Cognitive Resource Management

This lab implements ECAN (Economic Attention Network) for managing attention allocation across the SkinTwin cognitive architecture components.

### Key Concepts

- **STI (Short-Term Importance)**: Current relevance of a knowledge atom
- **LTI (Long-Term Importance)**: Persistent importance based on usage history
- **Attention Budget**: Total attention resources available for allocation
- **Spreading Activation**: Attention flows between connected atoms
- **Hebbian Learning**: Strengthens connections between co-activated atoms

### Architecture

```
┌─────────────────────────────────────────────────────────────┐
│                    ECAN Attention Manager                    │
├─────────────────────────────────────────────────────────────┤
│  Attention Budget │ Spreading │ Hebbian │ Forgetting        │
│   Allocation      │ Activation│ Learning│ Mechanism         │
├─────────────────────────────────────────────────────────────┤
│         Component Priority Weighting                        │
│  AtomSpace │ PLN │ MOSES │ ESN │ Agents                     │
└─────────────────────────────────────────────────────────────┘
```

In [ ]:
# Environment Setup
import os
import json
import numpy as np
from typing import List, Dict, Any, Optional, Set
from dataclasses import dataclass, field
from datetime import datetime
from collections import defaultdict
import heapq

from dotenv import load_dotenv
load_dotenv()

GATEWAY_URL = os.getenv('REGIMA_GATEWAY_URL', 'http://localhost:3000')
API_KEY = os.getenv('REGIMA_API_KEY', 'demo-key')

## 1. Attention Value Data Structures

In [ ]:
@dataclass
class AttentionValue:
    """Attention value for a knowledge atom"""
    sti: float = 0.0  # Short-term importance [-100, 100]
    lti: float = 0.0  # Long-term importance [0, 50]
    vlti: bool = False  # Very long-term importance (prevent forgetting)
    
    def total_importance(self) -> float:
        """Calculate total importance score"""
        return self.sti + self.lti
    
    def to_dict(self) -> Dict:
        return {
            "sti": round(self.sti, 4),
            "lti": round(self.lti, 4),
            "vlti": self.vlti,
            "total": round(self.total_importance(), 4)
        }

@dataclass
class AttentionAtom:
    """A knowledge atom with attention tracking"""
    atom_id: str
    atom_type: str
    name: str
    attention: AttentionValue = field(default_factory=AttentionValue)
    connections: Set[str] = field(default_factory=set)
    last_accessed: datetime = field(default_factory=datetime.now)
    access_count: int = 0
    
    def stimulate(self, amount: float):
        """Increase STI by given amount"""
        self.attention.sti = min(100, self.attention.sti + amount)
        self.last_accessed = datetime.now()
        self.access_count += 1
    
    def decay(self, rate: float):
        """Apply attention decay"""
        if not self.attention.vlti:
            self.attention.sti *= (1 - rate)
    
    def to_dict(self) -> Dict:
        return {
            "atom_id": self.atom_id,
            "atom_type": self.atom_type,
            "name": self.name,
            "attention": self.attention.to_dict(),
            "connections": list(self.connections),
            "access_count": self.access_count
        }

print("Attention data structures loaded")

## 2. ECAN Attention Manager

In [ ]:
class ECANAttentionManager:
    """Economic Attention Network for managing cognitive resources"""
    
    def __init__(
        self,
        attention_budget: int = 1000,
        spreading_rate: float = 0.3,
        decay_rate: float = 0.1,
        importance_threshold: float = 0.5,
        sti_cap: float = 100,
        lti_cap: float = 50,
        hebbian_rate: float = 0.05
    ):
        self.attention_budget = attention_budget
        self.available_budget = attention_budget
        self.spreading_rate = spreading_rate
        self.decay_rate = decay_rate
        self.importance_threshold = importance_threshold
        self.sti_cap = sti_cap
        self.lti_cap = lti_cap
        self.hebbian_rate = hebbian_rate
        
        self.atoms: Dict[str, AttentionAtom] = {}
        self.connection_weights: Dict[tuple, float] = defaultdict(lambda: 0.5)
        self.attention_focus: List[str] = []  # Current attention focus
        
        # Component allocation tracking
        self.component_budgets = {
            "atomspace": 0,
            "pln_reasoning": 0,
            "moses_optimization": 0,
            "esn_prediction": 0,
            "agent_responses": 0
        }
    
    def add_atom(self, atom: AttentionAtom):
        """Add an atom to the attention space"""
        self.atoms[atom.atom_id] = atom
    
    def connect_atoms(self, atom_id1: str, atom_id2: str, weight: float = 0.5):
        """Create bidirectional connection between atoms"""
        if atom_id1 in self.atoms and atom_id2 in self.atoms:
            self.atoms[atom_id1].connections.add(atom_id2)
            self.atoms[atom_id2].connections.add(atom_id1)
            key = tuple(sorted([atom_id1, atom_id2]))
            self.connection_weights[key] = weight
    
    def stimulate(self, atom_id: str, amount: float) -> Dict:
        """Stimulate an atom with attention"""
        if atom_id not in self.atoms:
            return {"error": f"Atom {atom_id} not found"}
        
        if amount > self.available_budget:
            amount = self.available_budget
        
        atom = self.atoms[atom_id]
        old_sti = atom.attention.sti
        atom.stimulate(amount)
        self.available_budget -= amount
        
        # Update attention focus
        if atom_id not in self.attention_focus:
            self.attention_focus.append(atom_id)
        
        return {
            "atom_id": atom_id,
            "old_sti": old_sti,
            "new_sti": atom.attention.sti,
            "amount_applied": amount,
            "remaining_budget": self.available_budget
        }
    
    def spread_attention(self) -> Dict:
        """Spread attention from high-STI atoms to connected atoms"""
        spreading_results = []
        
        # Get atoms above threshold, sorted by STI
        active_atoms = [
            a for a in self.atoms.values()
            if a.attention.sti > self.importance_threshold * self.sti_cap
        ]
        active_atoms.sort(key=lambda x: x.attention.sti, reverse=True)
        
        for atom in active_atoms:
            spread_amount = atom.attention.sti * self.spreading_rate
            
            if not atom.connections:
                continue
            
            # Spread to connected atoms
            share_per_connection = spread_amount / len(atom.connections)
            
            for conn_id in atom.connections:
                if conn_id in self.atoms:
                    conn = self.atoms[conn_id]
                    key = tuple(sorted([atom.atom_id, conn_id]))
                    weight = self.connection_weights[key]
                    
                    actual_spread = share_per_connection * weight
                    old_sti = conn.attention.sti
                    conn.attention.sti = min(self.sti_cap, conn.attention.sti + actual_spread)
                    
                    spreading_results.append({
                        "from": atom.atom_id,
                        "to": conn_id,
                        "amount": actual_spread,
                        "new_sti": conn.attention.sti
                    })
            
            # Reduce source atom STI
            atom.attention.sti -= spread_amount
        
        return {
            "spreading_events": len(spreading_results),
            "details": spreading_results[:10]  # Limit output
        }
    
    def apply_decay(self) -> Dict:
        """Apply attention decay to all atoms"""
        decay_results = []
        
        for atom in self.atoms.values():
            old_sti = atom.attention.sti
            atom.decay(self.decay_rate)
            
            if abs(old_sti - atom.attention.sti) > 0.01:
                decay_results.append({
                    "atom_id": atom.atom_id,
                    "old_sti": old_sti,
                    "new_sti": atom.attention.sti
                })
        
        # Return budget from decayed attention
        total_decayed = sum(r["old_sti"] - r["new_sti"] for r in decay_results)
        self.available_budget = min(self.attention_budget, self.available_budget + total_decayed * 0.5)
        
        return {
            "atoms_decayed": len(decay_results),
            "budget_recovered": total_decayed * 0.5,
            "available_budget": self.available_budget
        }
    
    def hebbian_update(self, atom_ids: List[str]) -> Dict:
        """Strengthen connections between co-activated atoms"""
        updates = []
        
        for i, id1 in enumerate(atom_ids):
            for id2 in atom_ids[i+1:]:
                if id1 in self.atoms and id2 in self.atoms:
                    key = tuple(sorted([id1, id2]))
                    old_weight = self.connection_weights[key]
                    
                    # Hebbian: strengthen connection
                    new_weight = min(1.0, old_weight + self.hebbian_rate)
                    self.connection_weights[key] = new_weight
                    
                    # Ensure connection exists
                    self.atoms[id1].connections.add(id2)
                    self.atoms[id2].connections.add(id1)
                    
                    updates.append({
                        "atoms": [id1, id2],
                        "old_weight": old_weight,
                        "new_weight": new_weight
                    })
        
        return {
            "connections_updated": len(updates),
            "updates": updates
        }
    
    def allocate_to_components(
        self,
        priorities: Dict[str, float],
        budget: int = None
    ) -> Dict:
        """Allocate attention budget to cognitive components"""
        budget = budget or self.available_budget
        
        # Normalize priorities
        total = sum(priorities.values())
        normalized = {k: v/total for k, v in priorities.items()}
        
        # Allocate budget
        allocation = {}
        for component, priority in normalized.items():
            allocated = int(budget * priority)
            allocation[component] = allocated
            self.component_budgets[component] = allocated
        
        self.available_budget -= sum(allocation.values())
        
        return {
            "total_allocated": sum(allocation.values()),
            "allocation": allocation,
            "remaining_budget": self.available_budget
        }
    
    def get_attention_focus(self, top_n: int = 10) -> List[Dict]:
        """Get top atoms by attention"""
        sorted_atoms = sorted(
            self.atoms.values(),
            key=lambda x: x.attention.total_importance(),
            reverse=True
        )[:top_n]
        
        return [a.to_dict() for a in sorted_atoms]
    
    def get_status(self) -> Dict:
        """Get current ECAN status"""
        return {
            "total_budget": self.attention_budget,
            "available_budget": self.available_budget,
            "total_atoms": len(self.atoms),
            "active_atoms": sum(
                1 for a in self.atoms.values()
                if a.attention.sti > self.importance_threshold
            ),
            "component_budgets": self.component_budgets,
            "attention_focus": self.get_attention_focus(5)
        }

print("ECAN Attention Manager loaded")

## 3. Initialize with Dermatology Knowledge

In [ ]:
# Initialize ECAN
ecan = ECANAttentionManager(
    attention_budget=1000,
    spreading_rate=0.3,
    decay_rate=0.1
)

# Add dermatology knowledge atoms
dermatology_atoms = [
    # Conditions
    AttentionAtom("cond_acne", "ConceptNode", "Acne Vulgaris", AttentionValue(lti=30, vlti=True)),
    AttentionAtom("cond_psoriasis", "ConceptNode", "Psoriasis", AttentionValue(lti=30, vlti=True)),
    AttentionAtom("cond_eczema", "ConceptNode", "Eczema", AttentionValue(lti=30, vlti=True)),
    AttentionAtom("cond_rosacea", "ConceptNode", "Rosacea", AttentionValue(lti=25, vlti=True)),
    
    # Treatments
    AttentionAtom("treat_retinoid", "ConceptNode", "Retinoid Therapy", AttentionValue(lti=25)),
    AttentionAtom("treat_steroid", "ConceptNode", "Topical Steroids", AttentionValue(lti=25)),
    AttentionAtom("treat_antibiotic", "ConceptNode", "Topical Antibiotics", AttentionValue(lti=20)),
    AttentionAtom("treat_moisturizer", "ConceptNode", "Emollient Therapy", AttentionValue(lti=20)),
    
    # Ingredients
    AttentionAtom("ing_benzoyl", "ConceptNode", "Benzoyl Peroxide", AttentionValue(lti=15)),
    AttentionAtom("ing_salicylic", "ConceptNode", "Salicylic Acid", AttentionValue(lti=15)),
    AttentionAtom("ing_niacinamide", "ConceptNode", "Niacinamide", AttentionValue(lti=15)),
    AttentionAtom("ing_ceramides", "ConceptNode", "Ceramides", AttentionValue(lti=15)),
    
    # Symptoms
    AttentionAtom("sym_inflammation", "ConceptNode", "Inflammation", AttentionValue(lti=20)),
    AttentionAtom("sym_dryness", "ConceptNode", "Dryness", AttentionValue(lti=15)),
    AttentionAtom("sym_redness", "ConceptNode", "Erythema", AttentionValue(lti=15)),
]

for atom in dermatology_atoms:
    ecan.add_atom(atom)

# Create knowledge connections
connections = [
    # Acne connections
    ("cond_acne", "treat_retinoid", 0.9),
    ("cond_acne", "treat_antibiotic", 0.7),
    ("cond_acne", "ing_benzoyl", 0.85),
    ("cond_acne", "ing_salicylic", 0.8),
    ("cond_acne", "sym_inflammation", 0.75),
    
    # Eczema connections
    ("cond_eczema", "treat_steroid", 0.85),
    ("cond_eczema", "treat_moisturizer", 0.9),
    ("cond_eczema", "ing_ceramides", 0.85),
    ("cond_eczema", "sym_dryness", 0.9),
    ("cond_eczema", "sym_inflammation", 0.7),
    
    # Rosacea connections
    ("cond_rosacea", "sym_redness", 0.9),
    ("cond_rosacea", "sym_inflammation", 0.8),
    ("cond_rosacea", "ing_niacinamide", 0.75),
    
    # Psoriasis connections
    ("cond_psoriasis", "treat_steroid", 0.8),
    ("cond_psoriasis", "sym_inflammation", 0.85),
    
    # Symptom-treatment connections
    ("sym_inflammation", "treat_steroid", 0.7),
    ("sym_dryness", "ing_ceramides", 0.8),
]

for id1, id2, weight in connections:
    ecan.connect_atoms(id1, id2, weight)

print(f"Initialized ECAN with {len(dermatology_atoms)} atoms and {len(connections)} connections")

## 4. Demonstrate Attention Operations

In [ ]:
# Check initial status
print("=" * 60)
print("INITIAL ECAN STATUS")
print("=" * 60)

status = ecan.get_status()
print(f"Total budget: {status['total_budget']}")
print(f"Available budget: {status['available_budget']}")
print(f"Total atoms: {status['total_atoms']}")
print(f"Active atoms: {status['active_atoms']}")

In [ ]:
# Simulate a clinical query about acne
print("\n" + "=" * 60)
print("SIMULATING ACNE QUERY")
print("=" * 60)

# Stimulate acne-related atoms
query_atoms = ["cond_acne", "sym_inflammation", "treat_retinoid"]

for atom_id in query_atoms:
    result = ecan.stimulate(atom_id, 50)
    print(f"\nStimulated {atom_id}:")
    print(f"  STI: {result['old_sti']:.2f} -> {result['new_sti']:.2f}")

print(f"\nRemaining budget: {ecan.available_budget}")

In [ ]:
# Spread attention to connected atoms
print("\n" + "=" * 60)
print("SPREADING ATTENTION")
print("=" * 60)

spread_result = ecan.spread_attention()
print(f"Spreading events: {spread_result['spreading_events']}")
print(f"\nSample spreading:")
for detail in spread_result['details'][:5]:
    print(f"  {detail['from']} -> {detail['to']}: +{detail['amount']:.2f} (new STI: {detail['new_sti']:.2f})")

In [ ]:
# Apply Hebbian learning
print("\n" + "=" * 60)
print("HEBBIAN LEARNING UPDATE")
print("=" * 60)

hebbian_result = ecan.hebbian_update(query_atoms)
print(f"Connections updated: {hebbian_result['connections_updated']}")
for update in hebbian_result['updates']:
    print(f"  {update['atoms'][0]} <-> {update['atoms'][1]}: {update['old_weight']:.2f} -> {update['new_weight']:.2f}")

In [ ]:
# Allocate to cognitive components
print("\n" + "=" * 60)
print("COMPONENT ALLOCATION")
print("=" * 60)

priorities = {
    "atomspace": 0.20,
    "pln_reasoning": 0.35,
    "moses_optimization": 0.25,
    "esn_prediction": 0.15,
    "agent_responses": 0.05
}

allocation = ecan.allocate_to_components(priorities, budget=500)
print(f"Total allocated: {allocation['total_allocated']}")
print(f"\nComponent allocations:")
for comp, amount in allocation['allocation'].items():
    print(f"  {comp}: {amount}")

In [ ]:
# Get current attention focus
print("\n" + "=" * 60)
print("CURRENT ATTENTION FOCUS")
print("=" * 60)

focus = ecan.get_attention_focus(10)
for i, atom in enumerate(focus, 1):
    print(f"{i}. {atom['name']}")
    print(f"   STI: {atom['attention']['sti']:.2f}, LTI: {atom['attention']['lti']:.2f}, Total: {atom['attention']['total']:.2f}")

In [ ]:
# Apply decay
print("\n" + "=" * 60)
print("APPLYING ATTENTION DECAY")
print("=" * 60)

decay_result = ecan.apply_decay()
print(f"Atoms decayed: {decay_result['atoms_decayed']}")
print(f"Budget recovered: {decay_result['budget_recovered']:.2f}")
print(f"Available budget: {decay_result['available_budget']:.2f}")

## 5. Gateway Integration

In [ ]:
import requests

class ECANGatewayClient:
    """Client for ECAN through RegimAI Gateway"""
    
    def __init__(self, gateway_url: str, api_key: str):
        self.gateway_url = gateway_url.rstrip('/')
        self.api_key = api_key
        self.headers = {
            "X-API-Key": api_key,
            "Content-Type": "application/json"
        }
    
    def allocate_attention(
        self,
        request_context: Dict,
        components: List[str],
        budget_limit: int = 500
    ) -> Dict:
        """Allocate attention for a request"""
        payload = {
            "request_context": request_context,
            "components": components,
            "budget_limit": budget_limit
        }
        
        response = requests.post(
            f"{self.gateway_url}/cognitive/ecan/allocate",
            headers=self.headers,
            json=payload
        )
        return response.json()
    
    def get_status(self) -> Dict:
        """Get ECAN status"""
        response = requests.get(
            f"{self.gateway_url}/cognitive/ecan/status",
            headers=self.headers
        )
        return response.json()
    
    def stimulate_atom(self, atom_id: str, amount: float) -> Dict:
        """Stimulate a specific atom"""
        payload = {
            "atom_id": atom_id,
            "amount": amount
        }
        
        response = requests.post(
            f"{self.gateway_url}/cognitive/ecan/stimulate",
            headers=self.headers,
            json=payload
        )
        return response.json()

print("ECAN Gateway Client ready")
print(f"Gateway URL: {GATEWAY_URL}")

## Summary

This lab implements:

1. **Attention Values**: STI/LTI tracking for knowledge atoms
2. **Spreading Activation**: Attention flows through connections
3. **Hebbian Learning**: Connection strengthening through co-activation
4. **Component Allocation**: Budget distribution across cognitive components
5. **Decay Mechanism**: Attention returns to pool over time

### Integration with SkinTwin

- **AtomSpace**: ECAN manages attention for knowledge atoms
- **PLN**: Higher attention = priority for reasoning
- **MOSES**: Attention guides evolutionary search focus
- **ESN**: Attention weights temporal predictions
- **Agents**: Response priority based on attention allocation